In [29]:
import requests
import json
import os
import pandas as pd
import textwrap

pd.set_option('display.max_colwidth', None)  # This will prevent truncating long text

# Function to collect user input for the health app. 
def collect_user_info():
    user_info = {}
    user_info['age'] = input("Enter your age: ")
    user_info['weight'] = input("Enter your weight (in kg): ")
    user_info['height'] = input("Enter your height (in cm): ")
    user_info['medical_conditions'] = input("Enter any known medical conditions (comma-separated): ")
    user_info['sedentary_hours'] = input("How many hours a day are you sedentary?: ")
    user_info['current_diet'] = input("Describe your current diet: ")
    user_info['sleep_hours'] = input("How many hours do you sleep daily?: ")
    user_info['physical_activity'] = input("Describe your current physical activity routine: ")
    user_info['stress_levels'] = input("Rate your stress levels from 1 (low) to 5 (high): ")   
    return user_info

# Function to send request to GPT-4o-mini and return a DataFrame with two fields: 'Factor' and 'Recommendations'
def get_health_plan(user_info):
    system_message = (
        "You are a health and wellness expert providing personalized weekly goals based on user input such as "
        "age, weight, height, medical conditions, sedentary hours, current diet, physical activity, stress levels, and sleep habits. "
        "For each recommendation, you also provide a specific explanation."
    )
    
    input_message = (
        f"Here is the user's health data: {json.dumps(user_info)}. "
        "Please suggest a weekly goal for physical activity, nutrition, and lifestyle changes. "
        "Each goal should combine the recommendation and the specific explanation into a single sentence or paragraph. "
        "In the explanation, refer to the given input to make to more explanable."
        "The addtional_notes should be specific, brief notes tailored for the user."
        "The output should be in the following JSON format:\n"
        "{{\n"
        "  'physical_activity': 'recommendation and explanation combined',\n"
        "  'nutrition': 'recommendation and explanation combined',\n"
        "  'lifestyle': 'recommendation and explanation combined',\n"
        "  'additional_notes': '...'\n"
        "}}"
    )

    # Set API key from environment variable
    api_key = os.getenv("OPENAI_KEY")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": input_message}
        ],
        "max_tokens": 8000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Parse the response and return a DataFrame
    if response.status_code == 200:
        response_data = response.json()
        output = response_data['choices'][0]['message']['content']
        try:
            # Convert the output to JSON format
            output_json = json.loads(output)
            
            # Create a DataFrame with 'Factor' and 'Recommendations' columns
            data = {
                "Factor": ["physical_activity", "nutrition", "lifestyle", "additional_notes"],
                "Recommendations": [
                    output_json.get('physical_activity', 'N/A'),
                    output_json.get('nutrition', 'N/A'),
                    output_json.get('lifestyle', 'N/A'),
                    output_json.get('additional_notes', 'None')
                ]
            }
            df = pd.DataFrame(data)
            return df
        
        except json.JSONDecodeError:
            print("Failed to parse the JSON response. Here is the raw output:")
            print(output)
            return None
    else:
        print(f"Request failed with status code {response.status_code}")
        print("Response Body:", response.text)
        return None


user_info = collect_user_info()  # Collect user information
df = get_health_plan(user_info)  # Get the DataFrame with health plan
df


Enter your age:  44
Enter your weight (in kg):  73
Enter your height (in cm):  170
Enter any known medical conditions (comma-separated):  diabetics
How many hours a day are you sedentary?:  8
Describe your current diet:  milk, eggs, porridge, vegetables, coffee, tea
How many hours do you sleep daily?:  7
Describe your current physical activity routine:  jogging for 20 minutes on every working day
Rate your stress levels from 1 (low) to 5 (high):  2


,Factor,Recommendations
0,physical_activity,"Aim to increase your jogging duration to 30 minutes on each working day to boost cardiovascular health and better manage your diabetes, as currently you are engaged in 20 minutes of jogging, which is beneficial but can be enhanced for greater fitness gains."
1,nutrition,"Incorporate more fiber-rich foods like legumes, whole grains, and fruits into your diet, while monitoring your carbohydrate intake, to help regulate blood sugar levels and provide sustained energy, since your current diet primarily includes milk, eggs, and porridge, which may not sufficiently support your diabetic management."
2,lifestyle,"Try to reduce your sedentary hours by integrating short activity breaks every hour, which can improve circulation and boost metabolism, especially since you're currently sitting for 8 hours and this can negatively impact your health over time."
3,additional_notes,"Consider keeping a food diary to track your carbohydrate intake, and prioritize getting at least 30 minutes of physical activity on weekends. Stay well-hydrated with water, and keep your stress levels low with mindfulness or relaxation techniques since you reported a low stress level."
